In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("adityajn105/flickr8k")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\janfa\.cache\kagglehub\datasets\adityajn105\flickr8k\versions\1


In [3]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
from nltk.translate.bleu_score import sentence_bleu
import json
import os

Set Up BLIP Model and Processor

In [4]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-0

In [8]:
image_path = path+"/Images/667626_18933d713e.jpg"  # Replace with an actual image filename
image = Image.open(image_path).convert("RGB")

In [9]:
inputs = processor(images=image, return_tensors="pt").to(device)

with torch.no_grad():
    output_ids = model.generate(**inputs)

generated_caption = processor.decode(output_ids[0], skip_special_tokens=True)
print("Generated Caption:", generated_caption)

c:\Users\janfa\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated Caption: a woman in a bikini on a surfboard


In [18]:
# Load the captions file
captions_file = path+"/captions.txt"  # Adjust this path if needed
actual_captions = {}

# Parse the captions file
with open(captions_file, 'r',encoding='utf-8') as f:
    for line in f:
        image_id, caption = line.strip().split(',', 1)  # Split by first comma only
        image_id = image_id.split('.')[0]  # Remove file extension for consistency
        actual_captions.setdefault(image_id, []).append(caption)

# Check if parsing is successful (prints sample)
print(actual_captions.get("667626_18933d713e", []))

['A girl is stretched out in shallow water', 'A girl wearing a red and multi-colored bikini is laying on her back in shallow water .', 'A little girl in a red swimsuit is laying on her back in shallow water .', '"A young girl is lying in the sand , while ocean water is surrounding her ."', 'Girl wearing a bikini lying on her back in a shallow pool of clear blue water .']


In [19]:
# Parse captions file
with open(captions_file, 'r') as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) == 2:
            image_id, caption = parts[0], parts[1]
            image_id = image_id.split('.')[0]
            actual_captions.setdefault(image_id, []).append(caption)

# Get actual captions for the selected image
image_id = os.path.basename(image_path).split('.')[0]
references = [caption.split() for caption in actual_captions.get(image_id, [])]

# Tokenize generated caption
candidate = generated_caption.split()

# Calculate BLEU score
bleu_score = sentence_bleu(references, candidate)
print("BLEU Score:", bleu_score)

BLEU Score: 9.696727898117387e-155


c:\Users\janfa\AppData\Local\Programs\Python\Python39\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\janfa\AppData\Local\Programs\Python\Python39\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
